In [0]:
### Common Utilities Notebook

# Author: Divij Kulshrestha
# Last updated: 2026-02-05
# Version: 1.1


### README
#### This notebook contains utility functions for data engineering workflows.
#### This notebook can be called by any other notebooks in the same Workspace
#### Use the %run magic command to call /Shared/utils at the start of your notebook to inherit these UDFs
#### You do not need to import the libraries that have already been called in this notebook
#### Certain parameters like file paths and secret keys are Workspace-specific. Please check before using in another Workspace.

In [0]:
#Pyspark Libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, IntegerType, TimestampType, DoubleType, StructType, StructField
)

#Python Libraries
import os
import uuid, time, traceback
from datetime import datetime
import re

# For IST Timezone
try:
    from zoneinfo import ZoneInfo
    _IST = ZoneInfo("Asia/Kolkata")
except Exception:
    try:
        import pytz
        _IST = pytz.timezone("Asia/Kolkata")
    except Exception:
        _IST = None

In [0]:
# Function to connect to ADLS
# "akp_connector" in Gera projects

def adls_connector(accountName, fileSystemName, idKey, secretKey, tenant_id):
    """
    Sets Spark Hadoop configuration entries to use OAuth (Client Credentials) for Azure ADLS Gen2.
    Builds and returns an ABFSS URI string
    
    Args:
        accountName: ADLS storage account name
        fileSystemName: Typically ADLS Gen2 file system name
        idKey: OAuth client ID
        secretKey: OAuth client secret
        tenant_id: OAuth tenant ID
    
    Returns:
        string: ABFSS URI string pointing to root in ADLS
    """
    spark.conf.set(f"fs.azure.account.auth.type.{accountName}.dfs.core.windows.net", "OAuth")
    spark.conf.set(f"fs.azure.account.oauth.provider.type.{accountName}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
    spark.conf.set(f"fs.azure.account.oauth2.client.id.{accountName}.dfs.core.windows.net", idKey)
    spark.conf.set(f"fs.azure.account.oauth2.client.secret.{accountName}.dfs.core.windows.net", secretKey)
    spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{accountName}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
    spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")
    return f"abfss://{fileSystemName}@{accountName}.dfs.core.windows.net"

In [0]:
import time
from datetime import datetime, timedelta
from typing import Optional, Dict, List
import pandas as pd
import builtins  # Import builtins to access original round function

class NotebookTimer:
    """Track execution time for notebook code blocks."""
    
    def __init__(self):
        self.blocks: Dict[str, Dict] = {}
        self.current_block: Optional[str] = None
        self.start_time: Optional[float] = None
        self.notebook_start: Optional[float] = None
        
    def start_notebook(self):
        """Start tracking the overall notebook execution."""
        self.notebook_start = time.time()
        print(f"📊 Notebook execution started at {datetime.now().strftime('%H:%M:%S')}")
        
    def start_block(self, block_name: str):
        """Start timing a new code block."""
        # End previous block if one is running
        if self.current_block:
            self.end_block()
            
        self.current_block = block_name
        self.start_time = time.time()
        
        if block_name not in self.blocks:
            self.blocks[block_name] = {
                'executions': [],
                'total_time': 0
            }
        
        print(f"\n⏱️  Starting: {block_name}")
        
    def end_block(self):
        """End timing the current code block."""
        if not self.current_block or not self.start_time:
            print("⚠️  No active block to end")
            return
            
        duration = time.time() - self.start_time
        self.blocks[self.current_block]['executions'].append(duration)
        self.blocks[self.current_block]['total_time'] += duration
        
        print(f"✅ Completed: {self.current_block} ({self._format_duration(duration)})")
        
        self.current_block = None
        self.start_time = None
        
    def end_notebook(self, output: str = "print"):
        """End notebook tracking and display summary.
        
        Args:
            output: "print" for console summary, "table" or "df" for DataFrame output,
                   "both" for both outputs
        """
        # End any active block
        if self.current_block:
            self.end_block()
            
        if not self.notebook_start:
            print("⚠️  Notebook timer was not started")
            return None
            
        total_time = time.time() - self.notebook_start
        
        # Print summary
        if output in ["print", "both"]:
            print("\n" + "="*60)
            print("📈 EXECUTION SUMMARY")
            print("="*60)
            
            # Display individual blocks
            for block_name, data in self.blocks.items():
                avg_time = data['total_time'] / len(data['executions'])
                pct = (data['total_time'] / total_time) * 100
                
                print(f"\n{block_name}:")
                print(f"  Duration: {self._format_duration(data['total_time'])}")
                print(f"  Percentage: {pct:.1f}%")
                if len(data['executions']) > 1:
                    print(f"  Executions: {len(data['executions'])} (avg: {self._format_duration(avg_time)})")
            
            print("\n" + "-"*60)
            print(f"🕐 Total Notebook Time: {self._format_duration(total_time)}")
            print("="*60)
        
        # Return DataFrame
        if output in ["table", "df", "both"]:
            return self.get_summary_df()
        
        return None
        
    def get_summary_df(self) -> pd.DataFrame:
        """Return execution summary as a DataFrame."""
        if not self.notebook_start:
            return pd.DataFrame()
            
        total_time = time.time() - self.notebook_start
        
        data = []
        for block_name, block_data in self.blocks.items():
            data.append({
                'Section': block_name,
                'Time Taken': self._format_duration(block_data['total_time']),
                'Time (seconds)': builtins.round(block_data['total_time'], 2),
                'Percentage': f"{(block_data['total_time'] / total_time) * 100:.1f}%"
            })
        
        # Add total row
        data.append({
            'Section': 'TOTAL',
            'Time Taken': self._format_duration(total_time),
            'Time (seconds)': builtins.round(total_time, 2),
            'Percentage': '100.0%'
        })
        
        df = pd.DataFrame(data)
        return df
    
    def _format_duration(self, seconds: float) -> str:
        """Format duration in human-readable format."""
        if seconds < 1:
            return f"{seconds*1000:.0f}ms"
        elif seconds < 60:
            return f"{seconds:.2f}s"
        else:
            minutes = int(seconds // 60)
            secs = seconds % 60
            return f"{minutes}m {secs:.1f}s"


# Create global timer instance
timer = NotebookTimer()

# Convenience functions
def start_notebook():
    """Start tracking notebook execution."""
    timer.start_notebook()

def start_block(name: str):
    """Start timing a code block."""
    timer.start_block(name)

def end_block():
    """End timing current code block."""
    timer.end_block()

def end_notebook(output: str = "print"):
    """End notebook and show summary.
    
    Args:
        output: "print" for console summary (default)
                "table" or "df" for DataFrame output
                "both" for both outputs
    """
    return timer.end_notebook(output)

def summary():
    """Display summary as DataFrame."""
    return timer.get_summary_df()


# USAGE EXAMPLE:
# ================

# At the start of your notebook:
# start_notebook()

# Before each code block:
# start_block("Setup")
# ... your setup code ...

# start_block("Load Data")
# ... your data loading code ...

# start_block("ETL Transforms - A")
# ... your ETL code ...

# start_block("ETL Transforms - B")
# ... more ETL code ...

# start_block("Write Data")
# ... your write code ...

# At the end of your notebook:

# Option 1: Print summary to console (default)
# end_notebook()

# Option 2: Get DataFrame for CSV export
# df = end_notebook("dataframe")
# df.to_csv("execution_report.csv", index=False)

# Option 3: Get both print and DataFrame
# df = end_notebook("both")
# df.to_csv("execution_report.csv", index=False)

# Option 4: Get DataFrame anytime
# df = summary()
# df.to_csv("execution_report.csv", index=False)

In [ ]:
import time
from contextlib import contextmanager

@contextmanager
def time_block(label: str):
    """
    Context manager to measure and log the elapsed execution time of a code block.

    This utility is intended for lightweight performance diagnostics and can be
    used in notebooks, batch jobs, Azure Functions, or any standard Python runtime.

    Parameters
    ----------
    label : str
        A descriptive name for the code block being measured. This label will
        be included in the log output to help identify the timed section.

    Yields
    ------
    None
        Control is yielded to the wrapped code block. Upon exit, the elapsed
        time is calculated and printed to standard output.

    Example
    -------
    >>> with time_block("load_exchange_rates"):
    ...     df = spark.read.table("exchange_rates")
    ...     df.count()

    Output
    ------
    [load_exchange_rates] Elapsed: 2.41s
    """
    start = time.time()
    try:
        yield
    finally:
        elapsed = round(time.time() - start, 2)
        print(f"[{label}] Elapsed: {elapsed}s")

In [0]:
class NotebookLogger:
    def __init__(self, log_filename="default_log.csv", path="/mnt/projects/logs/"):
        """
        Initialize a lightweight CSV logger for Databricks notebooks.
        
        Args:
            log_filename (str): Log file name (default: 'default_log.csv')
            path (str): ADLS mount path (default: '/mnt/projects/logs/')
        """
        self.path = path
        self.log_filename = log_filename
        self.full_path = os.path.join(path, log_filename)
        
        # Ensure the log file exists with header
        try:
            dbutils.fs.head(self.full_path)
        except:
            header = "timestamp,level,message\n"
            dbutils.fs.put(self.full_path, header, overwrite=True)
    
    def log(self, message, level="INFO"):
        """Append a timestamped log entry to the CSV file."""
       # Produce timestamp in IST (Asia/Kolkata). Fallback to local time if tz libs unavailable.
        if _IST is not None:
            timestamp = datetime.now(_IST).strftime("%Y-%m-%d %H:%M:%S %z")
        else:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        line = f"{timestamp},{level},{message}\n"
        #dbutils.fs.put(self.full_path, line, overwrite=False)
        try:
            existing = dbutils.fs.head(self.full_path)
        except:
            existing = ""
        dbutils.fs.put(self.full_path, existing + line, overwrite=True)
    
    def info(self, message):
        self.log(message, level="INFO")
    
    def warn(self, message):
        self.log(message, level="WARN")
    
    def error(self, message):
        self.log(message, level="ERROR")


In [0]:
# Function to coalesce and write a single CSV file to ADLS (not a folder)
def write_single_csv(df, base_path, base_filename, add_timestamp=True, container=None, storage_account=None, sep=",", header=True, overwrite=True):
    """
    Writes a Spark DataFrame as a single CSV file with optional timestamped name in ADLS (via abfss:// path).
    Will overwrite same file name by default.

    Example Usage: write_single_csv(df, ZONE2, "filename")
    
    Parameters:
    - df: Spark DataFrame
    - base_path: Path prefix (e.g., 'abfss://container@account.dfs.core.windows.net/intermediate')
    - base_filename: Base name for output file (e.g., 'buildingwise_combined')
    - add_timestamp: Whether to add timestamp to filename (default: True)
    - container, storage_account: (Optional) only needed if base_path is a relative ZONE path
    - sep: Field separator (default: ',')
    - header: Whether to include header (default: True)
    - overwrite: Overwrite mode (default: True)
    """

    # Generate filename with optional timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_filename = f"{base_filename}_{timestamp}.csv" if add_timestamp else f"{base_filename}.csv"
    temp_path = f"{base_path}/temp_{timestamp}"
    final_path = f"{base_path}/{final_filename}"

    # Write DataFrame as single-part CSV to temp folder
    writer = df.coalesce(1).write.option("sep", sep).option("header", str(header).lower())
    if overwrite:
        writer = writer.mode("overwrite")
    writer.csv(temp_path)

    # Find the single part file
    files = dbutils.fs.ls(temp_path)
    part_file = [f.path for f in files if f.name.startswith("part-")][0]

    # Move (rename) to final destination
    dbutils.fs.mv(part_file, final_path)

    # Clean up the temporary folder
    dbutils.fs.rm(temp_path, True)

    #return final_path

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.utils import AnalysisException
import traceback

def safe_read_csv(path: str, 
                  header=True, 
                  inferSchema=True, 
                  expected_cols: list[str] = None,  
                  mandatory=True) -> DataFrame:
    """
    Safely reads a CSV file with error handling and schema validation.
    Logs errors for empty files, schema mismatches, and read failures.
    Requires NotebookLogger implementation.
    
    Args:
        path (str): Full path of the CSV file.
        header (bool): Whether the CSV has headers. Default = True.
        inferSchema (bool): Whether to infer column types. Default = True.
        expected_cols (list): List of expected column names (optional).
        mandatory (bool): If True, raises an exception on failure instead of continuing.
    
    Returns:
        DataFrame: Spark DataFrame (empty if read fails).
    """
    try:
        df = spark.read.csv(path, header=header, inferSchema=inferSchema)
        if df.rdd.isEmpty():
            logger.warn(f"File at {path} is empty.")
            if mandatory:
                raise ValueError(f"Empty file at {path} (mandatory file).")
        
        if not all(col in df.columns for col in expected_cols):
            logger.warn(f"Unexpected schema in {path}: {df.columns}")
            if mandatory:
                raise ValueError(f"Unexpected schema in {path}: {df.columns}")  

        logger.info(f"Successfully read file: {path} with {df.count()} rows.")
        return df
    except AnalysisException as e:
        logger.warn(f"File not found or access issue at {path}: {str(e)}")
        if mandatory:
            raise e
    except Exception as e:
        logger.error(f"Error reading {path}: {traceback.format_exc()}")
        if mandatory:
            raise e

    # Return an empty DataFrame with no schema to allow continuation
    return spark.createDataFrame([], schema=None)

In [0]:
# Function to average non-zero rows in a column
def avg_nonzero(col_name, alias=None):
    """
    Return a Column that computes the average of non-zero values in a column.

    Zeros are treated as missing (excluded) by wrapping the column in a when(...).
    Existing nulls are ignored by pyspark.functions.avg. The resulting Column is
    aliased to `alias` (defaults to the original column name).

    Parameters
    ----------
    col_name : str
        Name of the column to average.
    alias : str, optional
        Alias for the resulting Column. If None, `col_name` is used.

    Returns
    -------
    pyspark.sql.Column
        A Column expression computing the average of non-zero values (null if no
        non-zero values are present).
    """    
    alias = alias or col_name
    return F.avg(F.when(F.col(col_name) != 0, F.col(col_name))).alias(alias)

In [0]:
# Spark - DataFrame Utilities

# Function to get dataframe details (rows, cols)
def get_shape(df) -> str:
    return f"rows: {df.count()} | cols: {len(df.columns)}"


# Return Dataframe Stats
def df_stats(df):
    return {
        "rows": df.count(),
        "partitions": df.rdd.getNumPartitions(),
        "schema": df.schema.simpleString()
    }

def check_column_exists(df, column_name: str) -> bool:
    """
    Check if a column exists in the given DataFrame
    
    Args:
        df: PySpark DataFrame to check
        column_name: Name of the column to look for (case-sensitive)
    
    Returns:
        bool: True if column exists, False otherwise
    """
    try:
        return column_name in df.columns
    except Exception as e:
        print(f"Error checking column {column_name}: {e}")
        return False
    
def compare_distinct_cardinality(df1, df2, col_name, col_name2=None):
    """
    Compare the distinct-value cardinality of a column between two DataFrames.

    Parameters
    - df1 (pyspark.sql.DataFrame): first DataFrame.
    - df2 (pyspark.sql.DataFrame): second DataFrame.
    - col_name (str): column name in df1 to compare.
    - col_name2 (str, optional): column name in df2 to compare. If None, col_name is used.

    Returns
    - bool: True if the number of distinct values in df1[col_name] equals the number in df2[col_name2], else False.

    Notes
    - Uses distinct().count(), which triggers Spark jobs and can be expensive on large datasets.
    - Column names are used as provided; pass col_name2 when the column name differs in df2 (e.g., "Year" vs "YEAR").
    """
    col2 = col_name if col_name2 is None else col_name2
    return df1.select(F.col(col_name)).distinct().count() == df2.select(F.col(col2)).distinct().count()

In [0]:

def underscore(word):
    """
    Make an underscored, lowercase form from the expression in the string.
    CamelCase words get undone into the lower_snake_case (standard for DataFrame columns in PySpark).

    Example::

        >>> underscore("DeviceType")
        'device_type'

    Inspired by PyPI Project Inflection maintained by jpvanhal.
    """
    word = re.sub(r"([A-Z]+)([A-Z][a-z])", r'\1_\2', word)
    word = re.sub(r"([a-z\d])([A-Z])", r'\1_\2', word)
    word = word.replace("-", "_")
    word = word.replace(" ", "_")
    return word.lower()


def standardize_columns(df):
    """
    Rename columns from CamelCase or Mixed Case into a standardized lower_snake_case.
    Column Names like "AvgHeight" or "Avg Height" will become "avg_height".
    """
    for c in df.columns:
        df = df.withColumnRenamed(c, inflection.underscore(c).lower())
    return df

In [ ]:

def log_event(run_id, pipeline_name, notebook_name, status, start_time=None, end_time=None, message=None):
    """
    Logs an event to a central admin/etl job run table.

    Requires a central Delta Table already created.
    
    Args:
        run_id (str): Can be parametrized to take from Azure Data Factory
        pipeline_name (str): Name of the pipeline (from Azure Data Factory)
        notebook_name (str): Name of the notebook (hard code in your notebook)
        status (str): Status of the event (STARTED/SUCCESS/FAILURE)
        start_time (datetime): Start time of the event (default: None)
        end_time (datetime): End time of the event (default: None)
        message (str): Message associated with the event (default: None)
        
    Returns:
        None
    """
    spark = SparkSession.getActiveSession()
    
    duration = None
    if start_time and end_time:
        duration = int(end_time.timestamp() - start_time.timestamp())

    #data = [(str(uuid.uuid4()), run_id, pipeline_name, notebook_name, status,
             #start_time, end_time, duration, message, None)]
    
    data = [(
        str(uuid.uuid4()),
        str(run_id),
        str(pipeline_name),
        str(notebook_name),
        str(status),
        start_time,   
        end_time,
        float(duration) if duration is not None else None,
        str(message) if message is not None else None,
        None
    )]
    
    columns = ["log_id", "run_id", "pipeline_name", "notebook_name", "status",
               "start_time", "end_time", "duration_sec", "message", "created_at"]
    
    schema = StructType([
            StructField("log_id", StringType(), False),
            StructField("run_id", StringType(), True),
            StructField("pipeline_name", StringType(), True),
            StructField("notebook_name", StringType(), True),
            StructField("status", StringType(), True),
            StructField("start_time", TimestampType(), True),
            StructField("end_time", TimestampType(), True),
            StructField("duration_sec", DoubleType(), True),
            StructField("message", StringType(), True),
            StructField("created_at", TimestampType(), True)
        ])
        
    df = spark.createDataFrame(data, schema)

    (df.withColumn("created_at", F.from_utc_timestamp(F.current_timestamp(), "Asia/Kolkata"))
       .withColumn("start_time", F.from_utc_timestamp(F.col("start_time"), "Asia/Kolkata"))
       .withColumn("end_time", F.from_utc_timestamp(F.col("end_time"), "Asia/Kolkata"))
       .write.format("delta")
       .mode("append")
       .save("/mnt/projects/logs/adf_run_logs_tbl"))
